In [1]:
'''
Date: 10 Apr 2020

Python version:      3.7
PyTorch version:     1.2.0

@author: Maksim Lavrov

CIFAR10 dataset

VGG16 model with one Flexible Layer in block1

Modified by Linnea Evanson 
09/01/2021
Used correct definition of flexible layer and baseline and calculated average accuracy to PGD attacks.

'''

'\nDate: 10 Apr 2020\n\nPython version:      3.7\nPyTorch version:     1.2.0\n\n@author: Maksim Lavrov\n\nCIFAR10 dataset\n\nVGG16 model with one Flexible Layer in block1\n\nModified by Linnea Evanson \n09/01/2021\nUsed correct definition of flexible layer and baseline and calculated average accuracy to PGD attacks.\n\n'

In [2]:
#Model
 #===================================================== Import libraries ================================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn 
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# ================================================= Flexible Layer ================================================================================

    
class FlexiLayer(nn.Module): # class FlexiLayer(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(FlexiLayer, self).__init__()
        
        self.t_1 = nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.t_2 = nn.MaxPool2d(self.kernel_size, self.stride, self.padding) # get max result with the same kernel size
        self.m = nn.Sigmoid()
        
        self.threshold1 = Variable(torch.randn((1, self.out_channels, 30, 30)))
        
        self.thresh_mean = []
        
    def forward(self, t):
        
        self.threshold1.expand(t.size(0), self.out_channels, 30, 30)
        
        
        cond = torch.sub(self.t_2(t), self.threshold1.cuda())
        t_2_2 = self.m(cond*50)*self.t_2(t) # 
        t_1_1 = self.m(cond*(-50))*self.t_1(t) # 
        t = torch.add(t_2_2, t_1_1)
        
        return t

    # ================================================= VGG-16 Network ================================================================================
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16,self).__init__()

        self.block1 = nn.Sequential(
                      nn.Conv2d(in_channels = 3,out_channels = 64,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(64),
                      nn.ReLU(),
                      FlexiLayer(in_channels = 64,out_channels = 64,kernel_size = 3, padding =0),
                      nn.BatchNorm2d(64),
                      nn.ReLU(),
                      #nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.3))

        self.block2 = nn.Sequential(
                      nn.Conv2d(in_channels = 64,out_channels = 128,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(128),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 128,out_channels = 128,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(128),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.4))

        self.block3 = nn.Sequential(
                      nn.Conv2d(in_channels = 128,out_channels = 256,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(256),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(256),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(256),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.4))

        self.block4 = nn.Sequential(
                      nn.Conv2d(in_channels = 256,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2) ,
                      nn.Dropout2d(0.4))

        self.block5 = nn.Sequential(
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.5) )

        self.fc =     nn.Sequential(
                      nn.Linear(512,100),
                      nn.Dropout(0.5),
                      nn.BatchNorm1d(100),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(100,10), )

    def forward(self,x):
        out = self.block1(x)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)

        return out


In [3]:
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms

from models.conv2model import *

parser = argparse.ArgumentParser(description='PyTorch CIFAR PGD Attack Evaluation')
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
parser.add_argument('--test-batch-size', type=int, default=100, metavar='N',
                    help='input batch size for testing (default: 200)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--epsilon', default=0.031,
                    help='perturbation')
parser.add_argument('--num-steps', default=20,
                    help='perturb number of steps')
parser.add_argument('--step-size', default=0.003,
                    help='perturb step size')
parser.add_argument('--random',
                    default=True,
                    help='random initialization for PGD')
parser.add_argument('--model-path',
                    default='./models/VGG16-flex-v5-block1-nnmodule_withdropoutmodel_150_90.pth',
                    help='model for white-box attack evaluation')
parser.add_argument('--source-model-path',
                    default='./models/conv2_model_80_74.pth',
                    help='source model for black-box attack evaluation')
parser.add_argument('--target-model-path',
                    default='./models/VGG16-flex-v5-block1-nnmodule_withdropoutmodel_150_90.pth',
                    help='target model for black-box attack evaluation')
parser.add_argument('--white-box-attack', default=False,
                    help='whether perform white-box attack')

args = parser.parse_args()

# settings
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# set up data loader
mean_cifar10 = [0.485, 0.456, 0.406]   
std_cifar10 = [0.229, 0.224, 0.225]
batch_size = 1

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10,std_cifar10),
])

testset = torchvision.datasets.CIFAR10(root='../FlexibleCNNs/data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=args.test_batch_size, shuffle=False, **kwargs)


def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=args.epsilon,
                  num_steps=args.num_steps,
                  step_size=args.step_size):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args.random:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(device)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    print('err pgd (white-box): ', err_pgd.item())
    return err, err_pgd


def _pgd_blackbox(model_target,
                  model_source,
                  X,
                  y,
                  epsilon=args.epsilon,
                  num_steps=args.num_steps,
                  step_size=args.step_size):
    out = model_target(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args.random:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(device)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()
        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model_source(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)

    err_pgd = (model_target(X_pgd).data.max(1)[1] != y.data).float().sum()
    #print('err pgd black-box: ', err_pgd.item())
    return err, err_pgd


def eval_adv_test_whitebox(model, device, test_loader):
    """
    evaluate model by white-box attack
    """
    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_whitebox(model, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    print('natural_err_total: ', natural_err_total)
    print('robust_err_total: ', robust_err_total)


def eval_adv_test_blackbox(model_target, model_source, device, test_loader):
    """
    evaluate model by black-box attack
    """
    model_target.eval()
    model_source.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_blackbox(model_target, model_source, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    print('natural_err_total: ', natural_err_total)
    print('robust_err_total: ', robust_err_total)
    
    return natural_err_total, robust_err_total


Files already downloaded and verified


In [4]:
# Main
pgd_naterror = []
pgd_roberror = []

for i in range(100): #do 100 iterations then find average robustness
    if args.white_box_attack:
        # white-box attack
        print('pgd white-box attack')
        model = VGG16().to(device)
        # Load the pretrained model
        state = torch.load('./models/VGG16-flex-v5-block1-model_150_90.pth')
        model.load_state_dict(state['model'])

        eval_adv_test_whitebox(model, device, test_loader)
    else:
        # black-box attack
        print('pgd black-box attack:', i)
        model_target = VGG16().to(device)
        state_target = torch.load(args.target_model_path)
        model_target.load_state_dict(state_target['model'])
        model_source = conv2model().to(device)
        state_source = torch.load(args.source_model_path)
        model_source.load_state_dict(state_source['model'])

        nat_err, rob_err = eval_adv_test_blackbox(model_target, model_source, device, test_loader)
        pgd_naterror.append(nat_err)
        pgd_roberror.append(rob_err)
        

pgd black-box attack: 0
natural_err_total:  tensor(997., device='cuda:0')
robust_err_total:  tensor(3940., device='cuda:0')
pgd black-box attack: 1
natural_err_total:  tensor(986., device='cuda:0')
robust_err_total:  tensor(3949., device='cuda:0')
pgd black-box attack: 2
natural_err_total:  tensor(980., device='cuda:0')
robust_err_total:  tensor(3901., device='cuda:0')
pgd black-box attack: 3
natural_err_total:  tensor(991., device='cuda:0')
robust_err_total:  tensor(3930., device='cuda:0')
pgd black-box attack: 4
natural_err_total:  tensor(990., device='cuda:0')
robust_err_total:  tensor(3950., device='cuda:0')
pgd black-box attack: 5
natural_err_total:  tensor(995., device='cuda:0')
robust_err_total:  tensor(3921., device='cuda:0')
pgd black-box attack: 6
natural_err_total:  tensor(984., device='cuda:0')
robust_err_total:  tensor(3910., device='cuda:0')
pgd black-box attack: 7
natural_err_total:  tensor(986., device='cuda:0')
robust_err_total:  tensor(3954., device='cuda:0')
pgd blac

natural_err_total:  tensor(1003., device='cuda:0')
robust_err_total:  tensor(3961., device='cuda:0')
pgd black-box attack: 67
natural_err_total:  tensor(997., device='cuda:0')
robust_err_total:  tensor(3880., device='cuda:0')
pgd black-box attack: 68
natural_err_total:  tensor(995., device='cuda:0')
robust_err_total:  tensor(3891., device='cuda:0')
pgd black-box attack: 69
natural_err_total:  tensor(1010., device='cuda:0')
robust_err_total:  tensor(3925., device='cuda:0')
pgd black-box attack: 70
natural_err_total:  tensor(992., device='cuda:0')
robust_err_total:  tensor(3932., device='cuda:0')
pgd black-box attack: 71
natural_err_total:  tensor(998., device='cuda:0')
robust_err_total:  tensor(3910., device='cuda:0')
pgd black-box attack: 72
natural_err_total:  tensor(987., device='cuda:0')
robust_err_total:  tensor(3921., device='cuda:0')
pgd black-box attack: 73
natural_err_total:  tensor(976., device='cuda:0')
robust_err_total:  tensor(3938., device='cuda:0')
pgd black-box attack: 7

In [5]:
#Calculate average Accuracy
# Accuracy = 10,000 - error.
#It is 10,000 because batch size is 100 images and each image is a %, so 100x100 = 10,000

avg_naterror = sum(pgd_naterror)/ len(pgd_naterror)
avg_natacc = (10000 - avg_naterror)/100
print("Average Nat. Acc = ", avg_natacc)

avg_roberror = sum(pgd_roberror)/ len(pgd_roberror)
avg_robacc = (10000 - avg_roberror)/100
print("Average Rob. Acc = ", avg_robacc)


Average Nat. Acc =  tensor(90.1179, device='cuda:0')
Average Rob. Acc =  tensor(60.7427, device='cuda:0')


In [7]:
#Save
import pickle
with open("pgd_blackbox_flex_avgrobacc.pkl", "wb") as output_file:
        pickle.dump(avg_robacc, output_file)

/home/levanson/anaconda3/envs/venv_clone/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
